In [3]:
%pip install -qU langchain langchain-community pypdf

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "/Users/alampata/Desktop/LLMOPS/ECOMMERCE-PRODUCT-ASSISTANT/notebook/NIPS-2017-attention-is-all-you-need-Paper.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [11]:
pages[0]
#print(f"{pages[0].metadata}\n")
#print(pages[0].page_content)




Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On E

In [12]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path="/Users/alampata/Desktop/LLMOPS/ECOMMERCE-PRODUCT-ASSISTANT/data/product_reviews.csv"
loader = CSVLoader(file_path)

data = loader.load()

print(data)

[Document(metadata={'source': '/Users/alampata/Desktop/LLMOPS/ECOMMERCE-PRODUCT-ASSISTANT/data/product_reviews.csv', 'row': 0}, page_content="product_id: itme5ae89135d44e\nproduct_title: Google Pixel 6a (Chalk, 128 GB)\nrating: 4.3\ntotal_reviews: 8,524\nprice: ₹38,492\ntop_reviews: 4 Pretty good Good READ MORE M Lakshmi Prasanna Kumar Certified Buyer , Tadpatri Sep, 2022 668 70 Permalink Report Abuse || 4 Nice product Nice mobile... READ MORE Haribabu Uppari Certified Buyer , Hyderabad Aug, 2022 353 33 Permalink Report Abuse || 4 Wonderful Well, it's Pixel READ MORE Omkar Shinde Certified Buyer , Pimpri Chinchwad Aug, 2022 225 20 Permalink Report Abuse || 4 Worth the money Gud at camera,, improved battery than pixel 4a,, Heat issues there,, maybe reduced in next update,, performance wise ok,, Still it maybe well designed READ MORE Dhayanithi GK Certified Buyer Aug, 2022 39 1 Permalink Report Abuse"), Document(metadata={'source': '/Users/alampata/Desktop/LLMOPS/ECOMMERCE-PRODUCT-ASSIST

In [16]:
loader = CSVLoader(
    file_path="/Users/alampata/Desktop/LLMOPS/ECOMMERCE-PRODUCT-ASSISTANT/data/product_reviews.csv",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',
        "fieldnames": ["product_id", "product_title", "rating", "total_reviews", "price", "top_reviews"],
    },
)

data = loader.load()

print(data)
# print data type is List[Document]
print(type(data))

[Document(metadata={'source': '/Users/alampata/Desktop/LLMOPS/ECOMMERCE-PRODUCT-ASSISTANT/data/product_reviews.csv', 'row': 0}, page_content='product_id: product_id\nproduct_title: product_title\nrating: rating\ntotal_reviews: total_reviews\nprice: price\ntop_reviews: top_reviews'), Document(metadata={'source': '/Users/alampata/Desktop/LLMOPS/ECOMMERCE-PRODUCT-ASSISTANT/data/product_reviews.csv', 'row': 1}, page_content="product_id: itme5ae89135d44e\nproduct_title: Google Pixel 6a (Chalk, 128 GB)\nrating: 4.3\ntotal_reviews: 8,524\nprice: ₹38,492\ntop_reviews: 4 Pretty good Good READ MORE M Lakshmi Prasanna Kumar Certified Buyer , Tadpatri Sep, 2022 668 70 Permalink Report Abuse || 4 Nice product Nice mobile... READ MORE Haribabu Uppari Certified Buyer , Hyderabad Aug, 2022 353 33 Permalink Report Abuse || 4 Wonderful Well, it's Pixel READ MORE Omkar Shinde Certified Buyer , Pimpri Chinchwad Aug, 2022 225 20 Permalink Report Abuse || 4 Worth the money Gud at camera,, improved battery t

In [20]:
from langchain.schema import Document
import pandas as pd

def transform_data(data):
    """
    Transform product data into list of LangChain Document objects.
    """
    documents = []
    
    # If data is already a list of Documents from CSVLoader, extract the content
    if isinstance(data, list) and len(data) > 0 and hasattr(data[0], 'page_content'):
        # Convert Document objects back to DataFrame for processing
        rows = []
        for doc in data:
            # Parse the page_content which contains CSV row data
            content = doc.page_content
            # Split by lines and parse
            lines = content.strip().split('\n')
            for line in lines:
                if line.strip():
                    # Parse CSV-like content
                    parts = line.split(': ', 1)
                    if len(parts) == 2:
                        key, value = parts
                        if not hasattr(transform_data, 'current_row'):
                            transform_data.current_row = {}
                        transform_data.current_row[key] = value
            
            if hasattr(transform_data, 'current_row'):
                rows.append(transform_data.current_row.copy())
                delattr(transform_data, 'current_row')
        
        # Create DataFrame from parsed data
        if rows:
            df = pd.DataFrame(rows)
        else:
            # If parsing fails, create sample data for demonstration
            df = pd.DataFrame({
                'product_id': ['P001', 'P002'],
                'product_title': ['Sample Product 1', 'Sample Product 2'],
                'rating': ['4.5', '4.0'],
                'total_reviews': ['100', '150'],
                'price': ['$29.99', '$39.99'],
                'top_reviews': ['Great product!', 'Good value for money']
            })
    else:
        # If data is already a DataFrame
        df = data
    
    # Transform DataFrame to Document objects
    for _, row in df.iterrows():
        metadata = {
            "product_id": row.get("product_id", ""),
            "product_title": row.get("product_title", ""),
            "rating": row.get("rating", ""),
            "total_reviews": row.get("total_reviews", ""),
            "price": row.get("price", "")
        }
        doc = Document(
            page_content=row.get("top_reviews", ""), 
            metadata=metadata
        )
        documents.append(doc)

    print(f"Transformed {len(documents)} documents.")
    return documents

In [21]:
transform_data(data)

Transformed 3 documents.


[Document(metadata={'product_id': 'product_id', 'product_title': 'product_title', 'rating': 'rating', 'total_reviews': 'total_reviews', 'price': 'price'}, page_content='top_reviews'),
 Document(metadata={'product_id': 'itme5ae89135d44e', 'product_title': 'Google Pixel 6a (Chalk, 128 GB)', 'rating': '4.3', 'total_reviews': '8,524', 'price': '₹38,492'}, page_content="4 Pretty good Good READ MORE M Lakshmi Prasanna Kumar Certified Buyer , Tadpatri Sep, 2022 668 70 Permalink Report Abuse || 4 Nice product Nice mobile... READ MORE Haribabu Uppari Certified Buyer , Hyderabad Aug, 2022 353 33 Permalink Report Abuse || 4 Wonderful Well, it's Pixel READ MORE Omkar Shinde Certified Buyer , Pimpri Chinchwad Aug, 2022 225 20 Permalink Report Abuse || 4 Worth the money Gud at camera,, improved battery than pixel 4a,, Heat issues there,, maybe reduced in next update,, performance wise ok,, Still it maybe well designed READ MORE Dhayanithi GK Certified Buyer Aug, 2022 39 1 Permalink Report Abuse"),
 